In [ ]:
from pprint import pprint
import pandas as pd

In [ ]:

from tf.app import use

A = use("etcbc/bhsa", hoist=globals())
Fmt = F
Lmt = L
Tmt = T

B = use("etcbc/dss", checkout="clone", version="1.8", hoist=globals())
Fdss = F
Ldss = L
Tdss = T

# here we rename the functions F L and T so the DSS functions does not overwrite the BHSA functions
# checkout="clone" is to look for the data set in the folder ~/github

What we are going to do now: for each occurrence of halak, we want to collect tf ID, book, chapter, verse, scroll name, stem, consonant rep. of the word, person number gender

otype: specifies the object type that we are looking for

In [ ]:
halak_info = {}

book_names = set()

for word in Fmt.otype.s("word"):
    if Fmt.lex.v(word) == "HLK[":
        book, chapter, verse = Tmt.sectionFromNode(word)
        book_names.add(book)
        stem = Fmt.vs.v(word)
        verbal_tense = Fmt.vt.v(word)
        consonant_rep = Fmt.g_cons.v(word)
        language = Fmt.language.v(word)
        features = [word, book, str(chapter), str(verse), "MT", stem, verbal_tense, consonant_rep, language]
        halak_info[word] = features

Structure in the DSS dataset: instead of book chapter verse being object type, they are features of word
book is a feature: word level feature, give the biblical book where the word occurs

In [ ]:
for word in Fdss.otype.s("word"):
    if Fdss.lex_etcbc.v(word) == "HLK[":
        book = Fdss.book_etcbc.v(word)
        chapter = Fdss.chapter.v(word)
        verse = Fdss.verse.v(word)
        scroll = Ldss.u(word, "scroll")[0]
        scroll_name = Tdss.scrollName(scroll)
        stem = Fdss.vs_etcbc.v(word)
        verbal_tense = Fdss.vt_etcbc.v(word)
        consonant_rep = Fdss.g_cons.v(word)
        language = Fdss.lang_etcbc.v(word)
        if book and book in book_names:   
            features = [word, book, chapter, verse, scroll_name, stem, verbal_tense, consonant_rep, language]
            halak_info[word] = features


In [ ]:
for word in Fdss.otype.s("word"):
    if Fdss.lex_etcbc.v(word) == "HLK[":
        book = Fdss.book_etcbc.v(word)
        chapter = Fdss.chapter.v(word)
        verse = Fdss.verse.v(word)
        print(word, book, chapter, verse)
        break

In [ ]:
pprint(halak_info)

In [ ]:
len(halak_info)

In [ ]:
halak_dataframe = pd.DataFrame(halak_info).T

In [ ]:
halak_dataframe.columns = ["word", "book", "chapter", "verse", "scroll_name", "stem", "verbal_tense", "consonant_rep", "language"]

In [ ]:
halak_dataframe.head()

In [ ]:
halak_dataframe.stem.value_counts()

In [ ]:
halak_dataframe.language.value_counts()

In [ ]:
pd.crosstab(halak_dataframe.stem, halak_dataframe.language)

In [ ]:
halak_dataframe.to_csv("halak_data.csv", sep="\t", index=False)

In [ ]:
pd.read_csv("halak_data.csv", sep="\t")